In [ ]:
import requests
import pandas as pd
import time

In [ ]:
def fetch_wikidata_stocks(ticker_list):
    url = 'https://query.wikidata.org/sparql'
    
    # Die SPARQL Query als Template (Fokus auf Englisch für maximale Daten)
    query_template = """
    SELECT ?ticker ?companyLabel ?isin ?foundingYear
      (GROUP_CONCAT(DISTINCT ?subLabel; separator="|") AS ?subsidiaries)
      (GROUP_CONCAT(DISTINCT ?prodLabel; separator="|") AS ?products)
      (GROUP_CONCAT(DISTINCT ?indLabel; separator="|") AS ?industries)
    WHERE {{
      VALUES ?ticker {{ {tickers_formatted} }}
      
      ?company p:P414 [ ps:P414 wd:Q82059 ; pq:P249 ?ticker ] .
      
      OPTIONAL {{ ?company wdt:P946 ?isin . }}
      OPTIONAL {{ ?company wdt:P571 ?fDate . BIND(YEAR(?fDate) AS ?foundingYear) }}
      OPTIONAL {{ ?company (wdt:P355|wdt:P1830) ?sub . ?sub rdfs:label ?subLabel . FILTER(LANG(?subLabel) = "en") }}
      OPTIONAL {{ ?company wdt:P1056 ?prod . ?prod rdfs:label ?prodLabel . FILTER(LANG(?prodLabel) = "en") }}
      OPTIONAL {{ ?company wdt:p452 ?ind . ?ind rdfs:label ?indLabel . FILTER(LANG(?indLabel) = "en") }}

      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    GROUP BY ?ticker ?companyLabel ?isin ?foundingYear
    """

    # Ticker für SPARQL formatieren: "AAPL" "MSFT" "TSLA"
    tickers_formatted = ' '.join([f'"{t}"' for t in ticker_list])
    query = query_template.format(tickers_formatted=tickers_formatted)

    # WICHTIG: User-Agent setzen (dein Name/Projekt)
    headers = {
        'User-Agent': 'PortfolioBot/1.0 (contact: deine-email@beispiel.de)',
        'Accept': 'application/sparql-results+json'
    }

    try:
        response = requests.get(url, params={'query': query, 'format': 'json'}, headers=headers)
        response.raise_for_status()
        data = response.json()

        # JSON in flache Liste umwandeln
        results = []
        for item in data['results']['bindings']:
            results.append({
                'ticker': item.get('ticker', {}).get('value'),
                'name': item.get('companyLabel', {}).get('value'),
                'isin': item.get('isin', {}).get('value'),
                'founded': item.get('foundingYear', {}).get('value'),
                'subsidiaries': item.get('subsidiaries', {}).get('value'),
                'products': item.get('products', {}).get('value'),
                'industries': item.get('industries', {}).get('value')
            })
        return results

    except Exception as e:
        print(f"Fehler bei Ticker-Set: {e}")
        return []

# --- BEISPIEL ANWENDUNG ---
meine_ticker = ["AAPL", "MSFT", "TSLA", "AMZN", "GOOGL", "NFLX", "NVDA"]

# In 20er Batches aufteilen, um Timeouts zu vermeiden
batch_size = 20
all_data = []

for i in range(0, len(meine_ticker), batch_size):
    current_batch = meine_ticker[i:i+batch_size]
    print(f"Lade Batch {i//batch_size + 1}...")
    batch_results = fetch_wikidata_stocks(current_batch)
    all_data.extend(batch_results)
    time.sleep(1) # Kurze Pause, um den Server zu schonen

# In Pandas DataFrame umwandeln und speichern
df = pd.DataFrame(all_data)
df.to_csv("aktien_analyse.csv", index=False)
print("Fertig! Daten in aktien_analyse.csv gespeichert.")

Lade Batch 1...


In [12]:
df

,ticker,name,isin,founded,subsidiaries,products,industries
0,NVDA,Nvidia,US67066G1040,1993,Ageia|GeForce Now|Icera|Mellanox Technologies|...,graphics processing unit|chipset|software,
1,TSLA,"Tesla, Inc.",US88160R1014,2003,Tesla Fremont Factory|Giga Nevada|Giga New Yor...,,
2,GOOGL,Alphabet Inc.,US02079K3059,2015,Google|Waymo|Google Fiber|X Development|Owlche...,,
3,NFLX,"Netflix, Inc.",US64110L1061,1997,Albuquerque Studios|Netflix Animation|Millarwo...,FAST Speed Test|FAST.com|Netflix,
4,AAPL,Apple Inc.,US0378331005,1976,"FileMaker, Inc.|Apple Store|Beats Electronics|...",HomePod|AirPods|Apple Pay|Apple Watch|consumer...,
5,MSFT,Microsoft,US5949181045,1975,Microsoft Global Finance|Microsoft Ireland Ope...,Microsoft Surface|Microsoft Dynamics|Azure|Mic...,
6,AMZN,Amazon,US0231351067,1994,Audible|Amazon Web Services|LoveFilm|Woot|Whol...,Amazon.com|Shopbop|Amazon Prime|Amazon Echo|Am...,
7,GOOGL,Google,None,1998,Boston Dynamics|Google Workspace|Google Ads|Vi...,search engine|Google Ads|Google Voice|Google S...,
